# Importimg necessary libs and datasets......

In [1]:
import pandas as pd
import numpy as np
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_excel("Titanic.xls")

# performing required pre-processing......

In [3]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
df = df.drop(columns=['Cabin'],axis=0)

In [5]:
df.isna().sum()/df.shape[0]

PassengerId    0.000000
Survived       0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.198653
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Embarked       0.002245
dtype: float64

In [6]:
df['Age'].value_counts()

24.00    30
22.00    27
18.00    26
19.00    25
30.00    25
         ..
55.50     1
70.50     1
66.00     1
23.50     1
0.42      1
Name: Age, Length: 88, dtype: int64

In [7]:
df = df.dropna()

In [8]:
df.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [9]:
df['Sex'].unique()

array(['male', 'female'], dtype=object)

In [10]:
df = df.reset_index()

In [11]:
df

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...,...
707,885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,Q
708,886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
709,887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
710,889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


In [12]:
df = df.drop(columns=['index','PassengerId','Name'])

In [13]:
df = df.drop(columns=['Ticket'])

In [14]:
df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  712 non-null    int64  
 1   Pclass    712 non-null    int64  
 2   Sex       712 non-null    object 
 3   Age       712 non-null    float64
 4   SibSp     712 non-null    int64  
 5   Parch     712 non-null    int64  
 6   Fare      712 non-null    float64
 7   Embarked  712 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 44.6+ KB


In [16]:
df['Pclass'] = df['Pclass'].astype('category')

In [17]:
df = pd.get_dummies(data=df,drop_first=True)

In [18]:
df

,Survived,Age,SibSp,Parch,Fare,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
0,0,22.0,1,0,7.2500,0,1,1,0,1
1,1,38.0,1,0,71.2833,0,0,0,0,0
2,1,26.0,0,0,7.9250,0,1,0,0,1
3,1,35.0,1,0,53.1000,0,0,0,0,1
4,0,35.0,0,0,8.0500,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...
707,0,39.0,0,5,29.1250,0,1,0,1,0
708,0,27.0,0,0,13.0000,1,0,1,0,1
709,1,19.0,0,0,30.0000,0,0,0,0,1
710,1,26.0,0,0,30.0000,0,0,1,0,0


In [19]:
df = df[['Sex_male', 'Age', 'SibSp', 'Parch', 'Fare','Pclass_2','Pclass_3',
       'Embarked_Q','Embarked_S','Survived']]

In [20]:
df = df.drop(columns=['Fare'])

In [22]:
df.columns

Index(['Sex_male', 'Age', 'SibSp', 'Parch', 'Pclass_2', 'Pclass_3',
       'Embarked_Q', 'Embarked_S', 'Survived'],
      dtype='object')

In [23]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# performing train-test split and smote.......

In [48]:
X,y = df.iloc[:,:-1],df.iloc[:,-1]
xtr,xts,ytr,yts = train_test_split(X,y,random_state=0,train_size=.7)

# Employing two ensmeble classifiers....

In [41]:
prmgrd = {
    'n_estimators':[100,150,200],
    'criterion':['gini','entropy'],
    'max_features':['auto','sqrt','log2'],
    'random_state':[123]
}
prmgrdx = {
        'n_estimators':[150,200,250,300],
        'booster':['gbtree','gblinear','dart']
}

In [42]:
gcvrf = GridSearchCV(estimator=RandomForestClassifier(),param_grid=prmgrd,n_jobs=-1)
gcvxg = GridSearchCV(estimator=XGBClassifier(),param_grid=prmgrdx,n_jobs=-1)

In [49]:
gcvrf.fit(xtr,ytr)
gcvxg.fit(xtr,ytr)

GridSearchCV(cv=None, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'booster': ['gbtree', 'gblinear', 'dart'],
                         'n_estimators': [150, 200, 250, 300]},
             pre_dispatch='2*n_jobs', refit=

In [50]:
gcvrf.best_params_,gcvxg.best_params_

({'criterion': 'entropy',
  'max_features': 'auto',
  'n_estimators': 200,
  'random_state': 123},
 {'booster': 'gbtree', 'n_estimators': 250})

In [51]:
gcvrf.score(xtr,ytr),gcvrf.score(xts,yts)
gcvxg.score(xtr,ytr),gcvxg.score(xts,yts)

(0.8955823293172691, 0.780373831775701)

In [52]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=122) 

In [62]:
for tri,tsi in skf.split(X,y):
  rfmd = RandomForestClassifier(criterion='entropy',max_features='auto',n_estimators=200,random_state=123,warm_start=True) 
  xgmd = XGBClassifier(booster='gbtree',n_estimators=250,warm_start=True) 
  xtri,ytri = X.iloc[tri],y.iloc[tri]
  xtsi,ytsi = X.iloc[tsi],y.iloc[tsi]
  rfmd.fit(xtri,ytri)
  xgmd.fit(xtri,ytri)
  print(rfmd.score(xtri,ytri),rfmd.score(xtsi,ytsi))
  print(xgmd.score(xtri,ytri),xgmd.score(xtsi,ytsi))

0.94375 0.8472222222222222
0.88125 0.8472222222222222
0.9453125 0.8333333333333334
0.8796875 0.8194444444444444
0.9516380655226209 0.7746478873239436
0.8814352574102964 0.8169014084507042
0.9422776911076443 0.8591549295774648
0.875195007800312 0.8591549295774648
0.9469578783151326 0.8169014084507042
0.8814352574102964 0.7605633802816901
0.9469578783151326 0.7464788732394366
0.8814352574102964 0.7605633802816901
0.9469578783151326 0.704225352112676
0.8783151326053042 0.8028169014084507
0.9469578783151326 0.7746478873239436
0.875195007800312 0.8028169014084507
0.953198127925117 0.7887323943661971
0.8767550702028081 0.8169014084507042
0.9516380655226209 0.7323943661971831
0.8876755070202809 0.7605633802816901


# Thus, Random Forest Classifier has better performance than XGBoost Classifier.....

In [63]:
print(rfmd.score(xtr,ytr),rfmd.score(xts,yts))
print(xgmd.score(xtr,ytr),xgmd.score(xts,yts))

0.9397590361445783 0.9065420560747663
0.8815261044176707 0.8598130841121495
